In [1]:
#image input

from PIL import Image
import numpy as np

img=Image.open('image.jpg')
rgb_image=img.convert("RGB")
rgb_array=(np.array(rgb_image)).tolist()


In [2]:
# Comparing color palette and ACI colors

#color pallete
RGB_dict={
9.5:[77,23,147],
10.5:[72,44,199],
11:[56,96,202],
12:[37,141,186],
12.5:[37,168,180],
13:[99,200,138],
13.5:[151,202,131],
14.5:[172,195,126],
15.5:[198,150,137],
16:[201,117,151],
16.5:[193,96,149],
17.5:[191,134,141],
18:[145,96,129],
19:[77,103,134],
0:[1,1,2]
}#Diameter-RGB data


def RGBerror(RGB1, RGB2):
    if (0.5*(RGB1[0]+RGB2[0])<128): return np.sqrt(2*((RGB1[0]-RGB2[0])**2)+4*((RGB1[1]-RGB2[1])**2)+3*((RGB1[2]-RGB2[2])**2))
    else: return np.sqrt(3*((RGB1[0]-RGB2[0])**2)+4*((RGB1[1]-RGB2[1])**2)+2*((RGB1[2]-RGB2[2])**2))


In [3]:
#calculating 2 RGB array's distance
def RGBerror(RGB1, RGB2):
    if (0.5*(RGB1[0]+RGB2[0])<128): return np.sqrt(2*((RGB1[0]-RGB2[0])**2)+4*((RGB1[1]-RGB2[1])**2)+3*((RGB1[2]-RGB2[2])**2))
    else: return np.sqrt(3*((RGB1[0]-RGB2[0])**2)+4*((RGB1[1]-RGB2[1])**2)+2*((RGB1[2]-RGB2[2])**2))


#choosing method; RGB color or W/B color
def color_method_decision(RGBdata):
    d1=RGBdata[0]-RGBdata[1]
    d2=RGBdata[2]-RGBdata[1]
    d3=RGBdata[2]-RGBdata[0]
    if (d1**2 + d2**2 + d3 **2 < 500): return "gray"
    else: return "RGB"


# Returning positions of RGB TIR circles
def mix_three_color_pointpos(pos, dim, lev, rgb_data):
    #startpoint(0,0), unit length of pixel, # of dot in line, ratio of colors - input 
    #RGB-positions array - output
    pointlist1=[]
    pointlist2=[]
    pointlist3=[]
    ratio_data=(np.array(rgb_data))
    randomlist=np.random.rand(lev*lev)
    randomlist_order=[]
    sortlist=sorted(randomlist)
    for i in randomlist:
        randomlist_order.append(sortlist.index(i))
    for i in range(lev):
        for j in range(lev):
            if(randomlist_order[i*lev+j]<ratio_data[0]*lev*lev):
                pointlist1.append([pos[0] + i*(dim/lev), pos[1] + j*(dim/lev)])
            elif(randomlist_order[i*lev+j]<ratio_data[0]*lev*lev+ratio_data[1]*lev*lev): 
                pointlist2.append([pos[0] + i*(dim/lev), pos[1] + j*(dim/lev)])
            elif(randomlist_order[i*lev+j]<ratio_data[0]*lev*lev+ratio_data[1]*lev*lev+ratio_data[2]*lev*lev): 
                pointlist3.append([pos[0] + i*(dim/lev), pos[1] + j*(dim/lev)])
    pointlist=[]
    pointlist.append(pointlist1)
    pointlist.append(pointlist2)
    pointlist.append(pointlist3)
    return pointlist

# Calculating average RGB from image's single pixel
def avg_lightlevel_RGB(img_array, posx, posy, dimx, dimy): 
    #배열, 시작점, 한 변 길이
    tot=np.array([0, 0, 0])
    temp=0
    for i in range(dimx):
        for j in range(dimy):
            if(img.size[0]>posx+i and img.size[1]>posy+j):
                tot=np.add(tot, np.array(img_array[posx+i][posy+j]))
            else: temp+=1
    return (tot/(dimx*dimy-temp)).tolist()

# Constructing combos of RGB
combo_list=[]
for i in RGB_dict.keys():
    for j in RGB_dict.keys():
        for k in RGB_dict.keys():
            if((i-j)*(j-k)*(k-i)!=0):
                if sorted([i,j,k]) not in combo_list:
                    combo_list.append(sorted([i,j,k]))

# fixing range
import copy
def ratio_fix(RGBdata):
    #RGB 0,0,0
    temp=copy.deepcopy(RGBdata)
    for i in range(3):
        if temp[i]<0:
            temp[i]=0
    sum=0
    for i in range(3): sum += temp[i]
    if sum!=0 and sum>1:
        for i in range(3): temp[i]=temp[i]/sum
    return temp

# returning RGB TIR dots' ratio from RGB data
def RGB_to_ratio(rgb, radius_list):
    #RGB (0,0,0), radius list(0,0,0)
    RGBmatrix=np.matrix([RGB_dict[radius_list[0]],RGB_dict[radius_list[1]],RGB_dict[radius_list[2]]])
    mat=np.linalg.inv(RGBmatrix)
    return ratio_fix(np.dot(mat, rgb).tolist()[0])    

def LABerror(LAB1, LAB2):
    return (LAB1[0]-LAB2[0])**2 + (LAB1[1]-LAB2[1])**2 + (LAB1[2]-LAB2[2])**2

def grayerror(RGB):
    average_brightness=int((RGB[0]+RGB[1]+RGB[2])/3)
    d1=RGB[0]-average_brightness
    d2=RGB[1]-average_brightness
    d3=RGB[2]-average_brightness
    return np.sqrt(3*float(d1**2+d2**2+d3**2))

# Finding the best combo
def RGBmixing(rgb, pixel):
    #RGB(0,0,0), # of dot in line
    best_combo=[0,0,0]
    err=1000000
    for Rlist in combo_list:
        ratio=RGB_to_ratio(rgb, Rlist)
        expected_RGB=np.dot((np.array(ratio)*pixel*pixel).astype(int)/(pixel*pixel), (RGB_dict[Rlist[0]],RGB_dict[Rlist[1]],RGB_dict[Rlist[2]]))
        RGBERR=RGBerror(rgb, expected_RGB)
        if(err>RGBERR):
            err=RGBERR
            best_combo=Rlist
    print(err)
    if(best_combo==[0,0,0]): return list(zip(best_combo,best_combo))
    return list(zip(best_combo,RGB_to_ratio(rgb, best_combo)))


# Positioning W/B TIR dots,  lev=4
def gray_pointpos(pos, dim, lev, rgb_data):
    pointlist=[]
    ratio_data=(rgb_data[0]+rgb_data[1]+rgb_data[2])/(3*256)
    randomlist=np.random.rand(lev*lev)
    randomlist_order=[]
    sortlist=sorted(randomlist)
    for i in randomlist:
        randomlist_order.append(sortlist.index(i))
    for i in range(lev):
        for j in range(lev):
            if(ratio_data*lev*lev > 1):
                if(randomlist_order[i*lev+j]<ratio_data*lev*lev):
                    pointlist.append([pos[0] + 3.4 + i*(32.4), pos[1] + 3.4 + j*(32.4)])
    return pointlist

In [4]:
# Class of single TIR pixel
class TIR_pixel:
    def __init__(self, RGBdata, position, length, dense_level):
        self.RGBdata=RGBdata
        self.position=(np.array(position)*length).tolist()
        self.length=length
        self.dense_level=dense_level
        self.color_method=color_method_decision(RGBdata)
        self.color_method="RGB"
        if(RGBdata[0]<20 and RGBdata[1]<20 and RGBdata[2]<20):
            self.color_method="gray"
        if(self.color_method=="RGB"):
            self.diameter_ratio_list=RGBmixing(self.RGBdata, self.dense_level)
            if([col[0]for col in self.diameter_ratio_list]==[0,0,0]):
                self.color_method="gray"
                self.point_list=[]
                self.gray_point_list=gray_pointpos(self.position, self.length, 4, self.RGBdata)
            else:
                self.point_list=mix_three_color_pointpos(self.position, self.length, self.dense_level, [col[1] for col in self.diameter_ratio_list])
                self.gray_point_list=[]
        elif(self.color_method=="gray"): 
            self.gray_point_list=gray_pointpos(self.position, self.length, 4, self.RGBdata)
            self.diameter_ratio_list=[]
            self.point_list=[]
        print(position)
        
    def RGB_array_return(self):
        temp_array=[[[0,0,0] for i in range(self.dense_level)]for j in range(self.dense_level)]
        if(self.color_method=="RGB"):
            for diameter_num in range(3):
                if(self.diameter_ratio_list[diameter_num][0]!=0):
                    for points in self.point_list[diameter_num]:
                        relative_pos=np.array(points)-np.array(self.position)
                        relative_pos_integer=(relative_pos/(self.length/self.dense_level)).astype(int)
                        temp_array[relative_pos_integer[0]][relative_pos_integer[1]]=RGB_dict[self.diameter_ratio_list[diameter_num][0]]
        elif(self.color_method=="gray"):
            mean_brightness=int((self.RGBdata[0]+self.RGBdata[1]+self.RGBdata[2])/3)
            mean_brightness=int(mean_brightness*17/(256))*int(230/16)
            return [[[mean_brightness, mean_brightness, mean_brightness] for i in range(self.dense_level)]for j in range(self.dense_level)]
        return temp_array


In [5]:
# Making RGB array after compressing
# Final image's size will be sampling_ratio x sampling_ratio
# Actual image's size in CAD will be (sampling_ratio*pixel_length)**2

sampling_ratio=280
unit_circle_level=5
pixel_length=5*26
samp_length=(img.size[0]/sampling_ratio)
Sampled_RGB_array=[[[0,0,0] for col in range(sampling_ratio)]for row in range(sampling_ratio)]
for x in range(sampling_ratio):
    for y in range(sampling_ratio):
        Sampled_RGB_array[x][y]=avg_lightlevel_RGB(rgb_array, int(x*samp_length), int(y*samp_length), int(samp_length), int(samp_length))

In [6]:
# Constructing array of TIR pixels

TIR_pixel_array=[[TIR_pixel(Sampled_RGB_array[x][y], [x,y], pixel_length, unit_circle_level) for y in range(sampling_ratio)]for x in range(sampling_ratio)]

2.4380319932273258
[0, 0]
2.7683930356797255
[0, 1]
[0, 2]
[0, 3]
4.711337389744017
[0, 4]
5.40182376609974
[0, 5]
4.5373450386762535
[0, 6]
4.017984569407903
[0, 7]
4.4073121060346985
[0, 8]
[0, 9]
4.857252309691151
[0, 10]
[0, 11]
[0, 12]
[0, 13]
2.7868082101213942
[0, 14]
[0, 15]
11.103377864415855
[0, 16]
11.093863168436862
[0, 17]
5.516330301930803
[0, 18]
1.4718016170666475
[0, 19]
3.8677125022421217
[0, 20]
1.7618740023055
[0, 21]
6.448015198493253
[0, 22]
9.240860349556193
[0, 23]
5.993738399363116
[0, 24]
4.465400318000617
[0, 25]
8.697413408594535
[0, 26]
4.749736834815186
[0, 27]
4.775730310643598
[0, 28]
8.043369940516223
[0, 29]
[0, 30]
[0, 31]
9.364368638621617
[0, 32]
6.642040349169829
[0, 33]
5.624419970094685
[0, 34]
9.978657224296262
[0, 35]
8.576648529583094
[0, 36]
5.524318962550948
[0, 37]
[0, 38]
3.1300638971113677
[0, 39]
5.817250209506207
[0, 40]
4.664740078503837
[0, 41]
11.557806885391365
[0, 42]
6.293139121297102
[0, 43]
8.784742454961314
[0, 44]
5.9226261067

In [7]:
# Exporting TIR circles to .dxf file

import ezdxf 
from ezdxf.gfxattribs import GfxAttribs

doc = ezdxf.new()
doc.layers.new("ENTITY")
msp = doc.modelspace()
attribs = GfxAttribs(layer="ENTITY")

for i in range(sampling_ratio):
    for j in range(sampling_ratio):
        if(TIR_pixel_array[i][j].color_method=="RGB"):
            for diameter_num in range(3):
                if(TIR_pixel_array[i][j].diameter_ratio_list[diameter_num][0]!=0):
                    for points in TIR_pixel_array[i][j].point_list[diameter_num]:
                        cursor=(points[0], points[1])
                        circle = msp.add_circle(cursor, radius=TIR_pixel_array[i][j].diameter_ratio_list[diameter_num][0]/2, dxfattribs=attribs)
                        circle.rgb=tuple(RGB_dict[TIR_pixel_array[i][j].diameter_ratio_list[diameter_num][0]])
        elif(TIR_pixel_array[i][j].color_method=="gray"):
            for points in TIR_pixel_array[i][j].gray_point_list:
                cursor=(points[0], points[1])
                circle = msp.add_circle(cursor, radius=13, dxfattribs=attribs)
                circle.rgb=(200,200,200)

doc.saveas("OUTPUT.dxf")